In [221]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from setup import path, SHOT_PATH, time_run_game
import pandas as pd
import importlib.util
import numpy as np
import sys
import json

SHOT_PATH = 'A:/AutoTrain/GAME/'

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHOT_PATH}base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

def load_module_player(player):
    return  importlib.util.spec_from_file_location('Agent_player', f"{SHOT_PATH}Agent/{player}/Agent_player.py").loader.load_module()

In [222]:
df_run = pd.read_json(f'{SHOT_PATH}Log/State.json')
df_agent = pd.read_json(f'{SHOT_PATH}Log/StateAgent.json')
df_env = pd.read_json(f'{SHOT_PATH}Log/StateEnv.json')

list_tong = list(df_run.index)
dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))
PASS_LEVEL = 250

In [223]:
df_run

,File,Size,Time1,Time2,STATE,ID,NOTE,Splendor_v3,MachiKoro,Splendor_v2,ngoxuanphong_1672396681,ngoxuanphong_1672398548
0,NaN,NaN,NaN,NaN,NaN,AgentBias1,NaN,DONE,DONE,DONE,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,AgentBias2,NaN,DONE,DONE,DONE,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,AgentBias3,NaN,DONE,DONE,DONE,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,AgentBias4,NaN,DONE,DONE,DONE,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,AgentBias5,NaN,DONE,DONE,DONE,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,AgentBias6,NaN,DONE,DONE,DONE,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,AgentBias7,NaN,DONE,DONE,DONE,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,AgentBias8,NaN,DONE,DONE,DONE,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,AgentBias9,NaN,DONE,DONE,DONE,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,AgentBias10,NaN,DONE,388,DONE,NaN,NaN


In [224]:
dict_level

{'Splendor_v3': {'Can_Split_Level': 'True',
  'level_max': 1,
  'id_remove': [10, 11, 0, 3, 1],
  '1': [[0, 3, 1],
   [5108.0, 5160.0, 5530.0],
   ['AgentBias1', 'AgentBias4', 'AgentBias2']]},
 'Splendor_v2': {'Can_Split_Level': 'True',
  'level_max': 1,
  'id_remove': [10, 11, 5, 0, 6],
  '1': [[5, 0, 6],
   [9310.0, 9328.0, 9422.0],
   ['AgentBias6', 'AgentBias1', 'AgentBias7']]},
 'MachiKoro': {'Can_Split_Level': 'True',
  'level_max': 3,
  'id_remove': [10, 11, 0, 4, 6, 8, 2, 3, 5, 7, 1],
  '1': [[0, 4, 6],
   [7533.0, 7646.0, 7688.0],
   ['AgentBias1', 'AgentBias5', 'AgentBias7']],
  '2': [[8, 2, 3],
   [509, 521, 586],
   ['AgentBias9', 'AgentBias3', 'AgentBias4']],
  '3': [[5, 7, 1],
   [336, 343, 412],
   ['AgentBias6', 'AgentBias8', 'AgentBias2']]},
 'ngoxuanphong_1672396681': {'Can_Split_Level': 'False',
  'level_max': 0,
  'id_remove': []},
 'ngoxuanphong_1672398548': {'Can_Split_Level': 'False',
  'level_max': 0,
  'id_remove': []}}

In [189]:
df_run['Splendor_v3'].iloc[0] = 5108.0
df_run['Splendor_v3'].iloc[1] = 5530.0
df_run['Splendor_v3'].iloc[3] = 5160.0

df_run['Splendor_v3'].iloc[4] = 5536.0
df_run['Splendor_v3'].iloc[5] = 5592.0
df_run['Splendor_v3'].iloc[7] = 5535.0

df_run.to_json(f'{SHOT_PATH}Log/State.json')

In [168]:
df_run.iloc[:, 7:]

,Splendor_v3,MachiKoro,Splendor_v2
0,DONE,7533.0,9328.0
1,DONE,8233.0,9814.0
2,5690.0,7990.0,9711.0
3,DONE,8072.0,9579.0
4,5536.0,7646.0,9616.0
5,5592.0,8868.0,9310.0
6,5705.0,7688.0,9422.0
7,5535.0,8353.0,9595.0
8,5649.0,8128.0,9811.0
9,5639.0,8532.0,9613.0


In [126]:
df_env['CHECK'] = np.nan
df_env['NOTE'] = np.nan
df_env.to_json(f'{SHOT_PATH}Log/StateEnv.json')

In [123]:
df_env['CHECK'] = np.nan
df_env['NOTE'] = np.nan
df_env.to_json(f'{SHOT_PATH}Log/StateEnv.json')

# Suawr laji maays file linh tinh vo van

In [82]:
# df_run = df_run.drop(labels=['TLMN', 'TLMN_v2'], axis=1)
# df_run = df_run.drop(11)
# df_run['Splendor_v3'] = np.nan
# df_run['Splendor_v2'] = np.nan
# df_run['MachiKoro'] = np.nan
# df_run.to_json(f'{SHOT_PATH}Log/State.json')